## Purpose
Annotates the pricing xlsx sheet with the inventory from wilco report.

It allows us to comput the value of 2 questions.
* What is the total dollar amount of WILCO inventory?
* What items are present in pricing sheet but not in wilco inventory? These can be manually checked to ensure that they exist at homte.

The final output for dollary amount is  *master_wilco_dollars.xlsx*

The various inputs are
* wilco inventory xlsx
* pricing xlsx to create a master list.
* lb inventory category

In [1]:
IN_DIR = "../private"
LB_CATEGORIES = "lb_inventory_categories.xlsx"


OUT_DIR = "out"
PRICING_XLSX = 'lb_sku_pricing.xlsx'
INVENTORY_XLSX = 'lb_sku_inventory.xlsx'

OUT_MASTER = "lb_inventory_master.xlsx"

In [2]:
import re
import os
import openpyxl
from pathlib import Path

import sys
sys.path.append("../src")
import lb_wilco_parser as lb


CATEGORIES_XLSX = Path(IN_DIR, LB_CATEGORIES)
IN_PRICING_XLSX = Path(OUT_DIR, PRICING_XLSX)
IN_INVENTORY_XLSX = Path(OUT_DIR, INVENTORY_XLSX)

OUT_XLSX = Path(OUT_DIR, OUT_MASTER)

if not os.path.exists(CATEGORIES_XLSX):
    raise ValueError("Categories not found")

if not os.path.exists(IN_PRICING_XLSX):
    raise ValueError("Pricing xlsx not found. Did you run the intermediate step?")

if not os.path.exists(IN_INVENTORY_XLSX):
    raise ValueError("Inventory xlsx not found. Did you run the intermediate step?")


In [3]:
wb = openpyxl.load_workbook(CATEGORIES_XLSX)
sheet = wb.active
categories = lb.create_categories_database(sheet)

Found 88 matching categories


In [4]:
print(categories)

{'001': ('Frozen', 'Parotta'), '003-20pk': ('Frozen', 'Tapioca'), '003-25pk': ('Frozen', 'Tapioca'), '004': ('Frozen', 'Coconut'), '010': ('Frozen', 'Coconut'), '022': ('Dry', 'Pickle'), '023': ('Dry', 'Pickle'), '026': ('Dry', 'Pickle'), '027': ('Dry', 'Pickle'), '028': ('Dry', 'Pickle'), '029': ('Dry', 'Pickle'), '036': ('Dry', 'Pickle'), '037': ('Dry', 'Pickle'), '038': ('Dry', 'Pickle'), '039': ('Dry', 'Pickle'), '041': ('Dry', 'Masala'), '042': ('Dry', 'Masala'), '044': ('Dry', 'Masala'), '048': ('Dry', 'Pickle'), '049SKU#: 36': ('Dry', 'Pickle'), '050': ('Dry', 'Pickle'), '054': ('Dry', 'Pickle'), '066': ('Dry', 'Snacks'), '068': ('Dry', 'Snacks'), '074': ('Dry', 'Masala'), '075': ('Dry', 'Pickle'), '076': ('Dry', 'Pickle'), '080': ('Dry', 'Pickle'), '083': ('Dry', 'Pickle'), '084': ('Dry', 'Pickle'), '086': ('Dry', 'Masala'), '087': ('Dry', 'Masala'), '088': ('Dry', 'Masala'), '089': ('Dry', 'Masala'), '090': ('Dry', 'Masala'), '091': ('Dry', 'Masala'), '092': ('Dry', 'Masala'),

In [5]:
wb = openpyxl.load_workbook(IN_INVENTORY_XLSX)
sheet = wb.active

In [6]:
inventory = {}
for row in range(2, sheet.max_row + 1):
    sku = str(sheet['A' + str(row)].value)
    description = sheet['B' + str(row)].value
    on_hand = sheet['C' + str(row)].value
    if sku in inventory:
        raise ValueError(f"Duplicate SKU {sku} found")
    inventory[sku] = (description, on_hand)
  

In [7]:
# Exception in inventory

#These SKU needs to be modified.
edit_keys = ['CP1', 'CP2', 'CP3']
del_keys = {}
for k in inventory.keys():
    for m  in edit_keys:
        if k is not m and k.startswith(m):
            del_keys[k] = m
            
for k in del_keys:
    temp = inventory[k]
    print(f"Updating {k} with {del_keys[k]}")
    inventory[del_keys[k]] = temp
    inventory.pop(k)
    

# SKU 115 Maryur Jaggery is not there even though inventory reports it.
inventory.pop('115')

# SKU 068: Tapioca Chips is not known. Ignore it
inventory.pop('068')

# SKU 083 Vadukapuli is not known.
inventory.pop('083')

# SKU 137-A Hand Grinder Ignore
inventory.pop('137-A')

# SKU 169 Coconut paste unkown
inventory.pop('169')

# SKU 026 Green Chilli Pickle
inventory.pop('026')

Updating CP1SKU#: 165,174,171,172 with CP1
Updating CP2SKU#: 168,166,170,167 with CP2
Updating CP3SKU#: 175,173,176,169 with CP3


('Green Chilli Pickle', 1)

In [9]:
wb = openpyxl.load_workbook(IN_PRICING_XLSX)
sheet = wb.active
pricing = {}
no_inventory_items = []
for row in range(2, sheet.max_row + 1):
    sku = str(sheet['A' + str(row)].value)
    if sku in inventory.keys():
        details = (sheet['C' + str(row)].value, sheet['D' + str(row)].value, sheet['E' + str(row)].value)
        if sku in pricing.keys():
            # Exceptions: Some items have same SKU but different pcs per case.
            # FIX: Use the smaller of the number.
            new_pcs_per_case = int(sheet['D' + str(row)].value)
            orig_pcs_per_case = int(pricing[sku][1])
            if new_pcs_per_case < orig_pcs_per_case:
                pricing[sku] = details
            print(f"Fixed duplicate SKU={sku} with {details}")
        else:
            pricing[sku] = details
    else:
        no_inventory_items.append(sku)

print(f">>Found SKU={no_inventory_items} that have no inventory but have pricing")
if len(no_inventory_items) > 0:
    print(f">>Skipping {len(no_inventory_items)} SKU from pricing sheet in the inventory")

Fixed duplicate SKU=094 with (100, 35, 1.36)
Fixed duplicate SKU=095 with (100, 35, 2.89)
Fixed duplicate SKU=116 with (400, 20, 2.82)
Fixed duplicate SKU=118 with (150, 20, 2.02)
>>Found SKU=['034', '032', '043', '044', '073', '051', '049', '081', '082', '057', '088', '091', '092', '093', '100', '101', '115', '115', '026', 'None'] that have no inventory but have pricing
>>Skipping 20 SKU from pricing sheet in the inventory


In [10]:
# Check if there are any inventory that does not have pricing
num_matches = 0
num_missing = 0
for sku in pricing.keys():
    if sku not in inventory.keys():
        print(f"SKU={sku} is in inventory but not in pricing")
        num_missing += 1
    else:
        num_matches += 1
        
print(f"Pricing=>Inventory:Missing {num_missing} entries and Matched {num_matches} entries")

Pricing=>Inventory:Missing 0 entries and Matched 79 entries


In [11]:
from openpyxl.styles import numbers

fmt_acc = u'$#,##0.00'

# Annotate the incoming WILCO report 
wb = openpyxl.Workbook()
sheet = wb.active

sheet['A1'].value = "SKU"
sheet['B1'].value = "Description"
sheet['C1'].value = "OnHand"
sheet['D1'].value = "Type"
sheet['E1'].value = "Category"
sheet['F1'].value = "wt_pc_in_gms"
sheet['G1'].value = "pcs_per_case"
sheet['H1'].value = "price_per_pc"
sheet['I1'].value = "price_per_case"
sheet['J1'].value = "total_value"

missing_sku_pricing = []
row = 1

for sku in inventory:
    row += 1
    description = sheet['B' + str(row)].value
    if not sku in pricing:
        print(f"Missing SKU={sku}, {description} in pricing")
        missing_sku_pricing.append(sku)
        continue
    details = pricing[sku]
    sheet['A' + str(row)].value = sku
    sheet['B' + str(row)].value = inventory[sku][0]
    sheet['C' + str(row)].value = inventory[sku][1]
    sheet['D' + str(row)].value = categories[sku][0]
    sheet['E' + str(row)].value = categories[sku][1]
    sheet['F' + str(row)].value = details[0]
    sheet['G' + str(row)].value = details[1]
    sheet['H' + str(row)].value = details[2]
    sheet['H' + str(row)].number_format = fmt_acc
    sheet['I' + str(row)].value = f"=G{row} * H{row}"
    sheet['I' + str(row)].number_format = fmt_acc
    sheet['J' + str(row)].value = f"=C{row} * I{row}"
    sheet['J' + str(row)].number_format = fmt_acc
    
wb.save(OUT_XLSX)

In [12]:
print(f"Found {len(missing_sku_pricing)} missing pricing info")

Found 0 missing pricing info


In [13]:
print(missing_sku_pricing)

[]
